In [113]:
import numpy as np
import pandas as pd

from datetime import datetime
from itertools import chain
from collections import defaultdict
from matplotlib_venn import venn2
import venn
import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt
import os


In [248]:
fit_min = 0.008


In [249]:
def join_conjunto_sets2(conjuntos, muertes):
    aux = []
    nuevo_conjunto = []
    siguiente_conjunto = 1
    tiempo = datetime.now()
    
    for A in conjuntos[:-1]:
        for B in conjuntos[siguiente_conjunto:]:
            if A - {max(A)} == B - {max(B)}:
                posible = A.union(B)
                muere = False
                aux.append(posible)
                for muerte in muertes:
                    if muerte.issubset(posible):
                        muere = True
                        break
                if not muere:
                    nuevo_conjunto.append(posible)
            else:
                continue

        siguiente_conjunto += 1    
    
    return nuevo_conjunto

In [250]:
def conjuntos_sobre_umbral_sets(cancion_esta_en, conjuntos, frec_min):
    
    sobrevivientes = []
    muertes = []
    
    diccionario = defaultdict(int)
    aux_cancion_esta_en = defaultdict(set)

    for conjunto in conjuntos:
        setlist = []
        canciones = []
        
        for cancion in conjunto:
            setlist.append(cancion_esta_en[cancion])
            canciones.append(cancion)
        
        repeticiones = len(set.intersection(*setlist))
        if repeticiones >= frec_min:
            sobrevivientes.append((conjunto, repeticiones))
        else:
            muertes.append(conjunto)

    return sobrevivientes, muertes

In [251]:
def fit(frec_min, datos='spotify.npy'):
    
    data = np.load(datos)
    #listas_datos = [set(x) for x in dict(data.item()).values()]
    
    listas_datos = {x[0]: set(x[1]) for x in dict(data.item()).items()}
    
    frec_min = frec_min * len(listas_datos)
    
    canciones = [x for songs in listas_datos.values() for x in songs]
    orden = list(set(canciones))

    enumeracion_canciones = dict(enumerate(orden))  # {0: cancion1, 1: cancion2}...
    canciones_enumeracion = {x[1]: x[0] for x in enumeracion_canciones.items()}  # {cancion1: 0, cancion2: 1... }
       
    playlist = {dato[0]: sorted([canciones_enumeracion[x] for x in dato[1]]) for dato in listas_datos.items()}



    '''####################
    
    data = {'0':['1', '2', '5'], '1':['2', '4'], '2':['2', '3'], '3':['1', '2', '4'], '4':['1', '3'], '4':['2', '3'], 
                                     '6':['1', '3'], '7':['1', '2','3', '5'], '8':['1', '2', '3']}
    #print(data)
    listas_datos = {x[0]: set(x[1]) for x in data.items()}
    # {0: {1, 2, 5}, 1: {2, 4}, 2: {2, 3}, 3: {1, 2, 4}, 4: {1, 3}, 5: {2, 3}, 6: {1, 3}, 7: {1, 2, 3, 5}, 8: {1, 2, 3}}
    frec_min = frec_min * len(listas_datos)

    canciones = [x for songs in listas_datos.values() for x in songs]
    orden = list(set(canciones))
    
    enumeracion_canciones = dict(enumerate(orden))  # {0: cancion1, 1: cancion2}...
    canciones_enumeracion = {x[1]: x[0] for x in enumeracion_canciones.items()}  # {cancion1: 0, cancion2: 1... }
        
    playlist = {dato[0]: sorted([canciones_enumeracion[x] for x in dato[1]]) for dato in listas_datos.items()}

    conjuntos = [{x} for x in canciones_enumeracion.values()]
    
    ####################'''
    

    # seleccionar elementos sobre el umbral  
    
    diccionario = defaultdict(int)
    total = chain(*playlist.values())
    

    t = datetime.now()
    for x in total:
        diccionario[x] += 1

    #print('diccionario frec tardo {} seg\n'.format((datetime.now() - t).total_seconds()))

    listas = [{x[0]} for x in (filter(lambda x: x[1] >= frec_min, diccionario.items()))]
    t = datetime.now()
    
    # seleccionar listas que posean elementos de interés
    cancion_esta_en = defaultdict(set) # {(canciones): [listas]}
    for indice, lista  in playlist.items():
        for conjunto in listas:
            if conjunto.issubset(lista):
                cancion_esta_en[min(conjunto)].add(indice)

    retorno = []    
    muerte = []
    aux = []
    for conjunto in listas:
        l = set()
        l.add(enumeracion_canciones[min(conjunto)])
        aux.append((l, len(cancion_esta_en[min(conjunto)])))

    retorno.extend(aux)

    while True: 
        t = datetime.now()
        
        conjuntos = join_conjunto_sets2(listas, muerte)
        listas, muerte = conjuntos_sobre_umbral_sets(cancion_esta_en, conjuntos, frec_min)

        repeticiones = [x[1] for x in listas]
        listas = [x[0] for x in listas]
        t = datetime.now()

        if not listas:  
            return retorno, len(listas_datos)
        
        else:
            aux = []
            for conjunto in range(len(listas)):
                l = set()
                for dato in listas[conjunto]:
                    l.add((enumeracion_canciones[dato]))
                aux.append((l, repeticiones[conjunto]))

            retorno.extend(aux)

In [ ]:
tiempo = datetime.now()
itemsets, len_datos = fit(fit_min)
print('tiempo:', (datetime.now() - tiempo).total_seconds())

In [ ]:
# ver el output 
for x in itemsets:
    #print(x)
    pass


In [ ]:
# subsets de un conjunto
def subsets(conjunto):
    retorno = []
    listas = list(conjunto)

    def backtrack(i=0):            
        if i == len(listas):
            yield set(retorno)
            
        else:
            yield from backtrack(i + 1)
            retorno.append(listas[i])
            
            yield from backtrack(i + 1)
            retorno.pop()

    yield from backtrack()

In [ ]:
# crea el archivo Resultados.csv con los conjuntos, subconjuntos y su confianza, esperanza y lift 
def generate(frecuentes, n):        
    table = []
    datos = []
    encabezado = ['conjunto', 'subconjunto', 'confianza', 'soporte', 'lift']
    conj_repetidos = []
    sub_repetidos = []


    for conjunto in sorted(frecuentes, key=lambda x: len(x[0])):  # dejo los itemsets de 1 elemento primero
        if len(conjunto[0]) == 1:
            if conjunto[0] in conj_repetidos:
                continue
            confianza = 1
            soporte = conjunto[1] / n        
            lift =  confianza / soporte
            table.append(['  |  '.join(list(conjunto[0])), '   - - -   ', str(round(confianza, 3)), str(round(soporte, 3)), str(round(lift, 3))])
            datos.append([conjunto[0], confianza, soporte, lift])

        else:
            elementos = conjunto[0] 
            frec = conjunto[1]
            subset = list(subsets(elementos)) 
                
            for conjunt in range(len(subset)): 
                if subset[conjunt]:
                    for A in elementos:
                        conj = elementos - {A}
                        sub = {A}
                        if conj in conj_repetidos and sub in sub_repetidos:
                            continue

                        conj_repetidos.append(conj)
                        sub_repetidos.append(sub)
                        
                        if conj:
                            frec_conj = 0
                            frec_sub = 0
                            for x in frecuentes:
                                if x[0] == sub:
                                    frec_sub = x[1]
                                if x[0] == conj:
                                    frec_conj = x[1]

                            confianza = conjunto[1] / frec_sub 
                            soporte = conjunto[1] / n

                            lift =  confianza / soporte
                            datos.append([elementos, sub, confianza, soporte, lift])
                            table.append(['  ;  '.join(list(elementos)) + ' ||  ', '   |  '.join(list(sub)), str(round(confianza, 3)), str(round(soporte, 5)), str(round(lift, 3))])

    with open('resultado.csv', 'w') as resultados:
        df =  pd.DataFrame(sorted(sorted(table, key=lambda x: x[3], reverse=True),key=lambda x: len((x[0].split(' ; '))), reverse=True), columns=encabezado)
        resultados.write(df.to_string(index=False))
            
    return datos
data = np.load('spotify.npy')  
listas_datos = [x for x in dict(data.item()).items()]
    
datos = generate(itemsets, len(listas_datos))


In [ ]:
def ranking(datos):
    # filtro 1: más de 1 item en el itemset
    itemsets = [x for x in datos if len(x) > 4]
    if not itemsets:
        return sorted([x for x in datos if x[-2] > 0.7 and x[-3] > 0.02], key=lambda x: x[-1], reverse=True)[:10]
    

    confianzas = sorted([x[-3] for x in itemsets])
    soportes = sorted([x[-2] for x in itemsets])    
    lift = sorted([x[-1] for x in itemsets])

    prom_conf = sum(confianzas) / len(confianzas)
    prom_sop = sum(soportes) / len(soportes)

    confianza_min = max((prom_conf + max(confianzas)) / 2, confianzas[round(0.75 * len(confianzas))])
    soporte_min = max((prom_sop + max(soportes)) / 2, soportes[round(0.75 * len(soportes))])
    
    n = 0
    iteracion = 0
    while n < 10 and n != len(itemsets): 
        if iteracion:
            confianza_min = confianza_min * 0.95
            soporte_min = soporte_min * 0.9
            
        # filtro 1:  conf > 0.9
        itemset_sobre_conf = [x for x in itemsets if x[-3] > confianza_min]

        # filtro 2: soporte > 0.012
        itemset_sobre_sop = [x for x in itemset_sobre_conf if x[-2] > soporte_min]

        # ordenar por lift 
        #top_10 = sorted(itemset_sobre_sop, key=lambda x: x[-1], reverse=True)[:10]

        top_10 = sorted(itemset_sobre_sop, key=lambda x: len(x[0]))[:10]
        
        n = len(top_10)
        iteracion += 1
    
    table = []
    encabezado = ['conjunto', 'subconjunto', 'confianza', 'soporte', 'lift']

    for x in top_10:
        table.append(['  |  '.join(list(x[0])), '   |  '.join(list(x[1])), str(round(x[2], 3)), str(round(x[3], 5)), str(round(x[4], 3))])

    with open('top.csv', 'w') as resultados:
        df =  pd.DataFrame(table, columns=encabezado)
        resultados.write(df.to_string(index=False))
    
    return top_10
        
rank = ranking(datos)


In [ ]:
for x in rank:
    print(x)

apariciones = [round(x[3] * 10000) for x in rank]
print(apariciones)

In [ ]:
def representar_confiaza(datos, len_data):

    data = len_data

    aux = [round(x[3] * 10000) for x in datos]

    retorno = []
    apariciones_conjunto = [(tuple(list(x[0].union(x[1]))), round(x[3] * data)) for x in datos]
    apariciones_subconjunto = [(tuple(list(rank[x][1])), round(aux[x] / datos[x][2])) for x in range(len(datos))]
    
    x = []
    y = []
    for k in apariciones_subconjunto:
        if k[0] not in x:
            x.append(k[0])
            y.append(k[1])
            
    for k in apariciones_conjunto:
        if k[0] not in x:
            x.append(k[0])
            y.append(k[1])

    fig, ax = plt.subplots()    
    width = 0.75 
    ind = np.arange(len(y)) 
    ax.barh(ind, y, width, color="blue")
    ax.set_yticks(ind+width/5)
    ax.set_yticklabels(x, minor=False)
    
    for i, v in enumerate(y):
        ax.text(v, i - 0.25, str(v))
        
    plt.title('Frecuencias relativas entre conjuntos')
    plt.xlabel('Apariciones')
    plt.ylabel('Conjuntos')      

    return apariciones_conjunto, apariciones_subconjunto
r_confianza = representar_confiaza(rank, 10000)

    

In [ ]:
def repr_confianza(apariciones_conjunto, apariciones_subconjunto):  
    for item in range(len(apariciones_conjunto)):
                conjunto = apariciones_conjunto[item][0]
                subconjunto = apariciones_subconjunto[item][0]
                n_conjunto = apariciones_conjunto[item][1]
                n_sub = apariciones_subconjunto[item][1]


                set_conjunto = set()
                set_sub = set()

                n = 0
                for x in range(n_sub):
                    set_sub.add(x)
                    if n < n_conjunto:
                        set_conjunto.add(x)
                    n += 1
                venn2([set_conjunto, set_sub], [', '.join(conjunto), ', '.join(subconjunto)])
                plt.show()
repr_confianza(*r_confianza)